# Форматы данных (2)

Материалы:
* Макрушин С.В. "Лекция 5: Форматы данных (часть 2)"
* https://docs.python.org/3/library/csv.html
* https://docs.h5py.org/en/stable/
* Уэс Маккини. Python и анализ данных

In [69]:
import csv
import json
import pickle
import pandas as pd
import numpy as np
import h5py

from bs4 import BeautifulSoup as bs
from pprint import pprint as pp


## Задачи для совместного разбора

1. Считайте данные из файла `open_pubs.csv`, используя `csv.reader`, и преобразуйте к структуре данных следующего вида:

`{'fas_id': [24, 30, ...], 'name': ['Achor Inn', 'Angel Inn', ...], ... }`

In [70]:
with open('./data/data/open_pubs.csv') as fp:
    reader = csv.reader(fp)
    header = next(reader)

    for row in reader:
        print(row)
        break

['24', 'Anchor Inn', 'Upper Street, Stratford St Mary, COLCHESTER, Essex', 'CO7 6LW', '604748', '234405', '51.97039', '0.979328', 'Babergh']


2. Сгенерируйте 2 случайные матрицы размера 10_000 x 10_000 и вычислите их произведение. Сколько времени занимают три этих операции? Сохраните 3 полученных матрицы в файл .npz с соответствующими названиями

In [71]:
A = np.random.randint(0, 100, size=(10_000, 10_000), dtype="int8")
B = np.random.randint(0, 100, size=(10_000, 10_000), dtype="int8")

In [72]:
np.save("A.npy", A)

In [73]:
A.dtype

dtype('int8')

In [74]:
# np.savez("AB.npz", artem=A, nikita=B)

In [75]:
r = np.load("AB.npz")
r

In [76]:
r.files

['artem', 'nikita']

In [77]:
r['artem']  #r['arr_0']

array([[37, 46, 70, ...,  6, 24, 99],
       [39,  2, 70, ..., 34, 48, 46],
       [55, 45, 88, ..., 84, 36, 34],
       ...,
       [30, 47, 45, ..., 80, 86, 75],
       [14, 81, 47, ..., 24, 89, 50],
       [61, 99, 49, ..., 53, 23, 14]], dtype=int8)

3. Создайте 2 матрицы размера 1000x1000, используя различные параметризируемые распределения из numpy (https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html#distributions)

После этого сохраните получившиеся матрицы в hdf5-файл в виде двух различных датасетов. В качестве описания каждого датасета укажите параметры используемых распределений

In [78]:
with h5py.File('./test.h5', 'w') as hdf:
    ds1 = hdf.create_dataset('arrA', data=A)
    ds2 = hdf.create_dataset('arrB', data=B)

    ds1.attrs["Description"] = "Здесь лежит массив А"
    ds2.attrs["Description"] = "Здесь лежит массив B"

OSError: [Errno 22] Unable to create file (unable to open file: name = './test.h5', errno = 22, error message = 'Invalid argument', flags = 13, o_flags = 302)

In [ ]:
with h5py.File('test.h5', 'r') as hdf:
    ds1 = hdf['arrA']
    print(type(ds1))
    arr = ds1[1500:5000]

In [ ]:
arr

## Лабораторная работа 5

### csv

1.1 В файле `tags_sample.csv` находится информация о тэгах, приписываемых рецептам. Воспользовавшись `csv.reader`, считайте этот файл и создайте словарь вида `id_рецепта: [список тэгов]`. Сохраните этот словарь в файл `tags_sample.json`.

In [ ]:
id_tags_dict = {}
with open("./data/data/tags_sample.csv") as fp:
    reader = csv.reader(fp)
    headers = next(reader)

    for row in reader:
        if row[0] not in id_tags_dict:
            id_tags_dict[row[0]] = list()
        id_tags_dict[row[0]].append(row[1])

with open("./data/data/tags_sample.json", 'w') as j:
    json.dump(id_tags_dict, j)

1.2 Считайте файл `recipes_sample_with_filled_nsteps.csv` (__ЛР4__) в виде `pd.DataFrame`. Добавьте к таблице 2 столбца: `n_tags`, содержащий количество тэгов у этого рецепта; и `tags`, содержащий набор тэгов в виде строки (тэги внутри строки разделяются символом `;`)

In [ ]:
# with open('../Lesson4-json_xml_pickle/data/data/recipes_sample_with_filled_nsteps.csv') as f:
#     reader = csv.reader(f)
#     header = next(reader)
#
#     for row in reader:
#         print(row)
#         break

df = pd.read_csv("../Lesson4-json_xml_pickle/data/data/recipes_sample_with_filled_nsteps.csv", index_col=0)
df.head()


In [ ]:
tags_series = pd.Series(id_tags_dict)
tags_df = pd.DataFrame(tags_series).reset_index()#, columns=["id", "tags"])
tags_df.columns = ['id', 'tags']
tags_df.head()


In [ ]:
tags_df["n_tags"] = tags_df["tags"].str.len()
tags_df.head()

In [ ]:
tags_df["tags"] = tags_df["tags"].apply(';'.join)
tags_df.head()

In [ ]:
recipes = pd.merge([df, tags_df], axis=1)

recipes.head()

In [ ]:

# x = recipes.drop(, 1)
# x.head()

In [ ]:
# recipes = recipes.drop(columns=["id"])
recipes.tail()

1.3 В файле `ingredients_sample.csv` находится информация о ингредиентах, необходимых для рецепта. Воспользовавшись `csv.DictReader`, считайте этот файл и создайте словарь вида `id_рецепта: [список ингредиентов]`.

In [ ]:
dictionary = {}
with open("./data/data/ingredients_sample.csv", "r") as csvf:
    reader = csv.DictReader(csvf)

    for row in reader:
        if row["recipe_id"] not in dictionary:
            dictionary[row["recipe_id"]] = []
        dictionary[row["recipe_id"]].append(row["ingredient"])

dictionary

1.4 Добавьте к таблице из задания 1.2 столбец `ingredients`, содержащий набор ингредиентов в виде строки (ингредиенты внутри строки разделяются символом `*`)

Для строк, которые содержат пропуски в столбце `n_ingredients`, заполните их на основе файла  `ingredients_sample.csv`

In [ ]:
# with open("./data/data/ingredients_sample.csv", 'r') as f:
ingredients_sample = pd.read_csv("./data/data/ingredients_sample.csv")
ingredients_sample.head()

In [ ]:
sr1 = pd.DataFrame(pd.Series(dictionary).reset_index())
sr1.columns= ["recipe_id", "ingredients"]
sr1["ingredients"] = sr1["ingredients"].apply('*'.join)
sr1.head()

In [ ]:
# recipes.head()
recipes = recipes.join(sr1)


In [ ]:
recipes.head()

In [ ]:
ingredients_sample.head()

In [ ]:
# out = pd.concat([ingredients_sample, sr1], axis=1)
# out.head()

In [ ]:
# recipes[np.isnan(recipes["n_ingredients"])]

In [ ]:
# ingredients_sample.head()

1.5 Проверьте, содержит ли столбец `n_ingredients` пропуски. Если нет, треобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_tags_ingredients.csv`

### npy

2.1 Разделите таблицу, полученную в результате 1.5, на две таблицы: одна содержит рецепты, загруженные до 2000 года; вторая - все остальные. В полученных таблицах оставьте только числовые столбцы и преобразуйте их к `numpy.array`

2.2. Сохраните 2 полученных массива в архив `npz`. Дайте массивам читаемые имена.

2.3 Считайте созданный архив и продемонстрируйте, что данные считались корректно. 

### hdf

3.1 Выведите названия всех датасетов, находящихся в файле `nutrition_sample.h5`, а также размерность матриц, содержащихся в данных датасетах и их метаданные.

Формат вывода:
```
Dataset name=dataset_0, dataset size=(30000,), metadata={'info': 'calories (#)'}
Dataset name=dataset_1, dataset size=(30000,), metadata={'info': 'total fat (PDV)'}
...
```

3.2 Разбейте каждый из имеющихся датасетов на две части: 1 часть содержит только те строки, где PDV (Percent Daily Value) превышает 100%; 2 часть содержит те строки, где PDV не составляет не более 100%. Создайте 2 группы в файле и разместите в них соответствующие части датасета c сохранением метаданных исходных датасетов. Итого должно получиться 2 группы, содержащие несколько датасетов. Сохраните результаты в файл `nutrition_grouped.h5`

3.3 Выведите названия всех групп и датасетов, находящихся в этих группах, из файла `nutrition_grouped.h5` а также размерность матриц, содержащихся в датасетах и их метаданные.

3.4 Модифицируйте код из 3.3 таким образом, чтобы сохранить датасеты, используя сжатие. Сравните размер полученного файла с размерами файла из 3.3. Прокомментируйте результат.